### Import packages

In [1]:
import os
import json

In [2]:
import sys
sys.path.append('../data_helpers')
sys.path.append('../preprocess/')

In [3]:
import numpy as np
import pandas as pd

In [4]:
from time import time, sleep
from tqdm import tqdm

In [5]:
from search_engine import SearchEngine

In [6]:
from text_cleaner import TextCleaner
text_cleaner = TextCleaner()

* [TextCleaner] Initializing...
* [TextCleaner] Loading SpaCy "en_core_web_md" corpus...
* [TextCleaner] Loading stopwords...
--------------------------------------------------------------------------------------------------------------------


In [7]:
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics

### Dataset

Annotated Corpus for Named Entity Recognition using GMB(Groningen Meaning Bank) corpus for named entity classification.

Type is List Object, to avoid unnecessary memory consumption while loading corpus.

This is the extract from GMB corpus which is tagged, annotated and built specifically to train the classifier to predict named entities such as name, location, etc.

Annotation scheme: [[((WORD, TAG), IOB), ...]...]
Each sublist and outer tuple represent each sentence and annotation for each word respectively.
Number of tagged entities:

'O': 1146068', geo-nam': 58388, 'org-nam': 48034, 'per-nam': 23790, 'gpe-nam': 20680, 'tim-dat': 12786, 'tim-dow': 11404, 'per-tit': 9800, 'per-fam': 8152, 'tim-yoc': 5290, 'tim-moy': 4262, 'per-giv': 2413, 'tim-clo': 891, 'art-nam': 866, 'eve-nam': 602, 'nat-nam': 300, 'tim-nam': 146, 'eve-ord': 107, 'per-ini': 60, 'org-leg': 60, 'per-ord': 38, 'tim-dom': 10, 'per-mid': 1, 'art-add': 1

Essential info about entities:

geo = Geographical Entity<br> org = Organization<br> per = Person<br> gpe = Geopolitical Entity<br> tim = Time indicator<br> art = Artifact<br> eve = Event<br> nat = Natural Phenomenon

Total Words Count = 1354149

In [ ]:
df = pd.read_csv('../dataset/ner/ner_dataset.csv', encoding='latin1')

In [ ]:
df

### Prepare person and place entity data

In [ ]:
df_per = pd.DataFrame()
df_per['Word'] = list(set(df[df['Tag'] == 'B-per']['Word']))
df_per['Tag'] = 'PERSON'

In [ ]:
df_geo = pd.DataFrame()
df_geo['Word'] = list(set(df[df['Tag'] == 'B-geo']['Word']))
df_geo['Tag'] = 'PLACE'

In [ ]:
df_data = pd.DataFrame()
df_data = pd.concat([df_per, df_geo])

In [ ]:
df_data.to_csv('../dataset/ner/ner_person_place_dataset.csv', encoding='utf-8', index=False)

### Randomly pick 200 persons and places

In [9]:
df_data = pd.read_csv('../dataset/ner/ner_person_place_dataset.csv')

In [10]:
num_person = 5000
num_place = 5000

person_idx = np.random.randint(low=0, high=len(df_data[df_data['Tag'] == 'PERSON']), size=num_person)
place_idx = np.random.randint(low=0, high=len(df_data[df_data['Tag'] == 'PERSON']), size=num_person)

df_person = df_data[df_data['Tag'] == 'PERSON'].iloc[person_idx]
df_place = df_data[df_data['Tag'] == 'PLACE'].iloc[place_idx]

df_person['Data'] = ''
df_place['Data'] = ''

### Let's retrieve 10 search results for each person and place

In [11]:
""" 如果有另一支程式在跑 search engine，最好不要用同一個 cache files，不然會打架 """
se = SearchEngine(verbose=True,
                  queries_filepath='../dataset/cache/queries2.txt',
                  output_filepath='../dataset/cache/output2.json')

In [12]:
person_keyed_df = se.get_data(queries=[word.encode('utf-8') for word in df_person['Word']])

[*] Writing queries to "../dataset/cache/queries2.txt"...
[*] Searching queries and save results to "../dataset/cache/output2.json"...
[*] Reading results from "../dataset/cache/output2.json"...
[*] Success.


In [13]:
place_keyed_df = se.get_data(queries=[word.encode('utf-8') for word in df_place['Word']])

[*] Writing queries to "../dataset/cache/queries2.txt"...
[*] Searching queries and save results to "../dataset/cache/output2.json"...
[*] Reading results from "../dataset/cache/output2.json"...
[*] Success.


### Assign search results (only text) to dataframe

In [14]:
for key in person_keyed_df.keys():
    df_person.loc[df_person['Word'] == key.split('b')[1].replace('\'', ''), 'Data'] = ' | '.join([text for text in person_keyed_df[key]['text']])
    
for key in place_keyed_df.keys():
    df_place.loc[df_place['Word'] == key.split('b')[1].replace('\'', ''), 'Data'] = ' | '.join([text for text in place_keyed_df[key]['text']])

In [15]:
df_data = pd.concat([df_person, df_place])
df_data.to_csv('../dataset/ner/ner_person_place_with_search_result_dataset.csv', encoding='utf-8', index=False)

### Split training data and test data

In [8]:
df_data = pd.read_csv('../dataset/ner/ner_person_place_with_search_result_dataset.csv')
df_data = df_data.fillna('')
df_data = df_data[df_data['Data'] != '']

data = df_data['Data'].tolist()
target = df_data['Tag'].tolist()
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.3, random_state=42)

In [10]:
df_data

,Word,Tag,Data
3,Izzadeen,PERSON,Understanding Muslim Identity: Rethinking Fund...
4,Jake,PERSON,Brother Jake - YouTube | Watch Brother Jake ba...
7,Coretta,PERSON,Coretta (Prod. Will. Quest) by MELLO B | Free ...
9,Oskar,PERSON,Oskar Blues Brewery - Wikipedia | Oskar Blues ...
10,Steve,PERSON,OH F**K YEAH BUD - B & STEVE FEAT. JACK MARKO ...
12,Rami,PERSON,B Rami Profiles | Facebook | View the profiles...
13,Deputy,PERSON,| Create fully costed staff rosters in no tim...
15,Taro,PERSON,RECKLI 1/124 B Taro | RECKLI - Design your con...
16,LeBron,PERSON,LeBron James - Wikipedia | LeBron Raymone Jame...
17,Margaret,PERSON,b margaret (@bharseim14) | Twitter | The lat...


In [9]:
print('PERSON class number in train: {}'.format(len([label for label in target_train if label == 'PERSON'])))
print('PLACE class number in train: {}'.format(len([label for label in target_train if label == 'PLACE'])))
print('PERSON class number in test: {}'.format(len([label for label in target_test if label == 'PERSON'])))
print('PLACE class number in test: {}'.format(len([label for label in target_test if label == 'PLACE'])))

PERSON class number in train: 2258
PLACE class number in train: 1414
PERSON class number in test: 967
PLACE class number in test: 607


### Entity Classfication using search results

### Transform feature vector

In [11]:
vectorizer = TfidfVectorizer(tokenizer=lambda text: text_cleaner.clean(texts=[text])[0], max_features=10000)

In [12]:
X_train = vectorizer.fit_transform(data_train)

1it [00:00, 23.99it/s]
1it [00:00, 30.74it/s]
1it [00:00, 30.52it/s]
1it [00:00, 28.91it/s]
1it [00:00, 43.01it/s]
0it [00:00, ?it/s]

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 31.68it/s]
1it [00:00, 32.91it/s]
1it [00:00, 32.31it/s]
1it [00:00, 33.43it/s]
1it [00:00, 34.20it/s]
1it [00:00, 44.43it/s]
1it [00:00, 33.66it/s]


* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 35.43it/s]
1it [00:00, 33.20it/s]
1it [00:00, 44.25it/s]
1it [00:00, 28.97it/s]
1it [00:00, 32.50it/s]
1it [00:00, 34.88it/s]
0it [00:00, ?it/s]

...

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 33.71it/s]
1it [00:00, 31.45it/s]
1it [00:00, 34.86it/s]
1it [00:00, 37.32it/s]
1it [00:00, 38.24it/s]
1it [00:00, 36.93it/s]
0it [00:00, ?it/s]

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 34.46it/s]
1it [00:00, 32.97it/s]
1it [00:00, 48.25it/s]
1it [00:00, 44.48it/s]


* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


In [13]:
X_test = vectorizer.transform(data_test)

1it [00:00, 31.56it/s]
1it [00:00, 41.17it/s]
1it [00:00, 32.16it/s]
1it [00:00, 49.15it/s]
1it [00:00, 31.46it/s]
1it [00:00, 37.28it/s]
0it [00:00, ?it/s]

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 34.69it/s]
1it [00:00, 47.65it/s]
1it [00:00, 42.58it/s]
1it [00:00, 73.86it/s]
1it [00:00, 32.98it/s]
1it [00:00, 30.86it/s]
1it [00:00, 40.82it/s]
1it [00:00, 47.78it/s]
0it [00:00, ?it/s]

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 42.21it/s]
1it [00:00, 40.50it/s]
1it [00:00, 33.46it/s]
1it [00:00, 36.49it/s]
1it [00:00, 32.52it/s]
1it [00:00, 40.08it/s]
0it [00:00, ?it/s]

...

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 26.12it/s]
1it [00:00, 26.76it/s]
1it [00:00, 29.45it/s]
1it [00:00, 29.76it/s]
1it [00:00, 45.02it/s]
1it [00:00, 33.16it/s]
0it [00:00, ?it/s]

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


1it [00:00, 41.21it/s]
1it [00:00, 35.32it/s]
1it [00:00, 29.37it/s]
1it [00:00, 78.60it/s]
1it [00:00, 47.39it/s]
1it [00:00, 53.52it/s]
1it [00:00, 45.74it/s]

* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...
* [TextCleaner] Cleaning text...


In [14]:
print(X_train.shape, X_test.shape)

(3672, 10000) (1574, 10000)


### Encode label to binary

In [15]:
label_encoder = LabelEncoder()
label_encoder.fit(target)

LabelEncoder()

In [16]:
y_train = label_encoder.transform(target_train)
y_test = label_encoder.transform(target_test)

In [17]:
print(y_train.shape, y_test.shape)

(3672,) (1574,)


### Try random forest model

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
t0 = time()

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

train_time = time() - t0
print("train time: %0.3fs" % train_time)

train time: 0.035s


### Test random forest performance

In [20]:
t0 = time()

y_pred = clf.predict(X_test)

test_time = time() - t0
print("test time:  %0.3fs" % test_time)

test time:  0.011s


In [21]:
score = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.621


### Show feature importance from random forest model

In [22]:
feature_names = vectorizer.get_feature_names()
feature_importances = clf.feature_importances_
indices = np.argsort(feature_importances)[::-1]
id2word = {i:feature_name for i, feature_name in enumerate(feature_names) }

for i in range(30):
    print('i: {} | feature id: {} | feature name: {} | score: {}'.format(i, indices[i], id2word[indices[i]], feature_importances[indices[i]]))

i: 0 | feature id: 9233 | feature name: tripadvisor | score: 0.07284662560408801
i: 1 | feature id: 7078 | feature name: power | score: 0.06763337283208773
i: 2 | feature id: 1525 | feature name: candid | score: 0.06461820635066193
i: 3 | feature id: 5167 | feature name: letter | score: 0.05439122887536293
i: 4 | feature id: 909 | feature name: bath | score: 0.054147164034578435
i: 5 | feature id: 7264 | feature name: province | score: 0.05363794810288679
i: 6 | feature id: 4924 | feature name: knowledge | score: 0.052973588996935994
i: 7 | feature id: 1510 | feature name: camp | score: 0.04813203852328278
i: 8 | feature id: 9197 | feature name: transport | score: 0.04680400603511104
i: 9 | feature id: 2405 | feature name: del | score: 0.046365421846464076
i: 10 | feature id: 3649 | feature name: give | score: 0.04612281463087471
i: 11 | feature id: 7036 | feature name: port | score: 0.04560877112463707
i: 12 | feature id: 5050 | feature name: landmark | score: 0.04180247169930423
i: 1

### Try neural net

In [23]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D
from keras.layers.pooling import AveragePooling1D, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [28]:
num_feature = X_train.shape[1]
model = Sequential()

model.add(InputLayer(input_shape=(num_feature,)))

model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1280128   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 1,280,257
Trainable params: 1,280,257
Non-trainable params: 0
_________________________________________________________________


### Train neural net

In [36]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model.fit(X_train.toarray(), y_train, validation_split=0.33, epochs=100, callbacks=[early_stopping])
model.fit(X_train.toarray(), y_train, validation_split=0.33, epochs=100, callbacks=[early_stopping])

Train on 2460 samples, validate on 1212 samples
Epoch 1/100
2460/2460 [==============================] - 2s - loss: 0.6483 - acc: 0.6146 - val_loss: 0.6099 - val_acc: 0.6221
Epoch 2/100
2460/2460 [==============================] - 2s - loss: 0.5122 - acc: 0.7772 - val_loss: 0.4932 - val_acc: 0.7929
Epoch 3/100
2460/2460 [==============================] - 2s - loss: 0.3448 - acc: 0.8911 - val_loss: 0.4352 - val_acc: 0.8284
Epoch 4/100
2460/2460 [==============================] - 2s - loss: 0.2399 - acc: 0.9207 - val_loss: 0.4201 - val_acc: 0.8342
Epoch 5/100
2460/2460 [==============================] - 2s - loss: 0.1768 - acc: 0.9435 - val_loss: 0.4290 - val_acc: 0.8366
Epoch 6/100
2460/2460 [==============================] - 2s - loss: 0.1382 - acc: 0.9516 - val_loss: 0.4367 - val_acc: 0.8243
Epoch 7/100
2460/2460 [==============================] - 2s - loss: 0.1184 - acc: 0.9577 - val_loss: 0.4569 - val_acc: 0.8226
Epoch 8/100
2460/2460 [==============================] - 2s - loss: 0.

### Test neural net
We have significant perfomance boost!

In [38]:
score = model.evaluate(X_test.toarray(), y_test, batch_size=X_test.shape[0])[1]
print("accuracy:   %0.3f" % score)

1574/1574 [==============================] - 0s
accuracy:   0.812
